In [1]:
import os
import sys

os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.1-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

In [2]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate() #getOrCreate -> 세션이 이미 있으면 불러오고 아니면 생성한다

In [3]:
myRdd2 = spark.sparkContext\
    .textFile(os.path.join("data", "ds_spark_wiki.txt"))

In [4]:
myRdd2.take(10)

[u'Wikipedia',
 u'Apache Spark is an open source cluster computing framework.',
 u'\uc544\ud30c\uce58 \uc2a4\ud30c\ud06c\ub294 \uc624\ud508 \uc18c\uc2a4 \ud074\ub7ec\uc2a4\ud130 \ucef4\ud4e8\ud305 \ud504\ub808\uc784\uc6cc\ud06c\uc774\ub2e4.',
 u'Apache Spark Apache Spark Apache Spark Apache Spark',
 u'\uc544\ud30c\uce58 \uc2a4\ud30c\ud06c \uc544\ud30c\uce58 \uc2a4\ud30c\ud06c \uc544\ud30c\uce58 \uc2a4\ud30c\ud06c \uc544\ud30c\uce58 \uc2a4\ud30c\ud06c',
 u"Originally developed at the University of California, Berkeley's AMPLab,",
 u'the Spark codebase was later donated to the Apache Software Foundation,',
 u'which has maintained it since.',
 u'Spark provides an interface for programming entire clusters with',
 u'implicit data parallelism and fault-tolerance.']

In [6]:
myRdd_group = myRdd2.groupBy(lambda x:x[0:2])
# 위의 groupBy함수를 이용하면 () 안에 해당되는 인자가 key가 되어 이를 바탕으로 분류해준다.

for (k, v) in myRdd_group.collect(): #groupBy를 통하여 나온 결과는 (key, value)로
    for eachValue in v: #v는 그룹된 인자(문장단위이다.)
        print k, eachValue #eachValue는 하나의 문장 
    print "-----"  #출력 결과는 key eachValue가 v안에 있는 만큼 계속 나온 뒤 ----으로 분류

 아파 아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.
아파 아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크
-----
im implicit data parallelism and fault-tolerance.
-----
th the Spark codebase was later donated to the Apache Software Foundation,
-----
Wi Wikipedia
-----
Ap Apache Spark is an open source cluster computing framework.
Ap Apache Spark Apache Spark Apache Spark Apache Spark
-----
Sp Spark provides an interface for programming entire clusters with
-----
Or Originally developed at the University of California, Berkeley's AMPLab,
-----
wh which has maintained it since.
-----


In [8]:
_testList=[("key1",1),("key1",1),("key1",1),("key2",1),("key2",1),
           ("key1",1),("key2",1),
           ("key1",1),("key1",1),("key2",1),("key2",1)] 
#key1, key2로 명확히 key값이 분류된 정형 데이터로 이해해보자.
_testRdd=spark.sparkContext.parallelize(_testList)

In [11]:
_testRdd.groupBy(lambda x:x[0]).collect() 
#x[0]을 키값으로 사용
#이 또한 출력결과가 Rdd이므로 Iterable

[('key2', <pyspark.resultiterable.ResultIterable at 0x6f66908>),
 ('key1', <pyspark.resultiterable.ResultIterable at 0x6f66940>)]

In [12]:
_testRdd.groupBy(lambda x:x[0]).mapValues(lambda x:list(x)).collect()
#mapvalues(lambda x:list(x))를 통해 groupBy Rdd 고유의 type을 list type으로 변환

[('key2', [('key2', 1), ('key2', 1), ('key2', 1), ('key2', 1), ('key2', 1)]),
 ('key1',
  [('key1', 1),
   ('key1', 1),
   ('key1', 1),
   ('key1', 1),
   ('key1', 1),
   ('key1', 1)])]